In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from load_data import load_data
from sklearn.model_selection import train_test_split
import numpy as np


# 创建模型序列
model = Sequential()
#输入的图片为227x227， RGB通道
#第一层卷积网络，使用96个大小为11x11的卷积核，步幅为4，激活函数使用relu
model.add(Conv2D(96, (11, 11), strides=(1, 1), input_shape=(227, 227, 3), padding='same', activation='relu',
                 kernel_initializer='uniform'))
#尺寸缩小到(55x55x96)，缩小接近4倍

# 3x3最大池化层
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
# 尺寸变成27x27x96

# 第二层使用256个5x5的卷积核，加边，激活函数为relu
model.add(Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
#尺寸变为27×27×276

#使用3x3池化层，步长为2
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
#尺寸变成13×13

# 第三层卷积，大小为3x3的卷积核使用384个
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
#尺寸变为13x13x384

# 第四层卷积,同第三层
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# 第五层卷积使用的卷积核为256个，其他同上
model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))

#执行一次最大池化
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
#尺寸缩小到6×6×256 9216个元素

#全连接层
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
#分类
model.add(Dense(20, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()


checkpoint = ModelCheckpoint('AlexNet.hdf5', verbose = 1, monitor = 'val_accuracy', save_best_only = True)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


Using TensorFlow backend.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 227, 227, 96)      34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 113, 113, 96)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 113, 113, 256)     614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 384)       885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 384)       1327488   
_________________________________________________________________
conv2d_5 (Conv2D)           

In [2]:
# 载入数据
path = './Standard/RGB227'
X,y = load_data(path,128,layers=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, shuffle = True, random_state = 666)

[('Pikachu', 454), ('Squirtle', 410), ('Bulbasaur', 387), ('Mewtwo', 355), ('Charmander', 354), ('Psyduck', 264), ('Raichu', 174), ('Spearow', 168), ('Muk', 161), ('Aerodactyl', 152), ('Dratini', 151), ('Fearow', 143), ('Shellder', 126), ('Rapidash', 107), ('Mankey', 103), ('Ninetales', 101), ('Sandslash', 99), ('Lapras', 98), ('Pidgey', 97), ('Machamp', 88)]
Class1:Pikachu
Class2:Squirtle
Class3:Bulbasaur
Class4:Mewtwo
Class5:Charmander
Class6:Psyduck
Class7:Raichu
Class8:Spearow
Class9:Muk
Class10:Aerodactyl
Class11:Dratini
Class12:Fearow
Class13:Shellder
Class14:Rapidash
Class15:Mankey
Class16:Ninetales
Class17:Sandslash
Class18:Lapras
Class19:Pidgey
Class20:Machamp
(3992, 227, 227, 3)


In [4]:
model.fit(X_train, y_train, batch_size=32, epochs=10,validation_data = [X_test, y_test],callbacks = [checkpoint])
score = model.evaluate(X_test, y_test, batch_size=32)

Train on 3193 samples, validate on 799 samples
Epoch 1/10
  32/3193 [..............................] - ETA: 4:15:37 - loss: 3.9832 - accuracy: 0.0000e+00